# Recomendador basado en contenidos
Consiste en recomendar contenido similar a un determinado contenido

#### Principales caracteristicas
- Sólo necesita la información de un producto, por lo que no se necesita información histórica
- Se suele usar para evitar problemas de arranque en frio

#### Como lo hacemos
1. Definir una métrica de similitud para comparar los contenidos
2. Calcular la similitud entre todos los elementos con la métrica seleccionada
3. (Opcional) Definir vecindarios para optimizar


## Antes de comenzar... veamos el dataset
Recursos turisticos de Andalucia obtenidos de [TripAdvisor](https://www.tripadvisor.es)

In [1]:
import pandas as pd

recursos = pd.read_csv('./recursos/Recursos_turisticos.csv', sep='|')
recursos.head(5)

,_id,nombre_ws,strCategoria,strSubcategoria,strCiudad,strDescripcion
0,5880b148636e564ad8d9b952,Hotel Albucasis,Alojamientos,Hoteles,Córdoba,.El hotel tiene a disposición de sus clientes ...
1,5880b148636e564ad8d9b963,Hotel Sierra Hidalga,Alojamientos,Hoteles,Ronda,Enclavado en una situación privilegiada con fa...
2,5880b1f2636e564ad8d9b969,Hotel Virgen de los Reyes,Alojamientos,Hoteles,Sevilla,"El Hotel Virgen de los Reyes***, ubicado en el..."
3,5880b1f2636e564ad8d9b96a,Hotel An Santo Domingo,Alojamientos,Hoteles,Lucena,El Hotel Santo Domingo Lucena está situado en ...
4,5880b1f2636e564ad8d9b96f,Leflet Valme,Alojamientos,Hoteles,Dos Hermanas,El Hotel Leflet Valme está situado estratégica...


## Metrica de similitud para texto

Una de las técnicas más utilizadas para obtener similitude texto es TF-IDF, tecnica proveniente del campo de Information Retrieval.

TF-IDF permite realizar una representación vectorial de los textos. Por ejemplo:

> "En un lugar de la mancha"

|_|En|un |lugar|de|la|mancha|
|--|--|---|-----|--|--|------|
|TF |1|1|1|1|1|1|
|DF |200|1700|100|3000|4075|3|
|IDF|1/200|1/1700|1/100|1/3000|1/4075|1/3|

Al tener una representación vectorial de los textos podemos usar medidas tradicionales
<img src="./recursos/similitudes.png">

### Ejemplo TF-IDF

**Paso 1** Preparacion y tokenizado de datos

In [2]:
import gensim
import nltk
import tempfile
import heapq

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

documentos = recursos['strDescripcion']
print ("Numero de documentos:", len(documentos))
print ("\n", documentos[1])

#Tokenización
docs_tokenizado = [[w for w in word_tokenize(text)] 
             for text in documentos]
print ("\n\nElementos tokenizados\n")
print (docs_tokenizado[1])

# Convertimos a minúsculas
docs_tokenizado = [[w.lower() for w in documento] 
             for documento in docs_tokenizado]
print ("\n\nTodo minúsculas\n")
print (docs_tokenizado[1])

#Quitamos puntaciones
import string
table = str.maketrans('', '', string.punctuation)
docs_tokenizado = [[w.translate(table) for w in documento] 
             for documento in docs_tokenizado]
docs_tokenizado = [[w for w in documento if w.isalpha()] 
             for documento in docs_tokenizado]

print ("\n\nPuntuaciones\n")
print (docs_tokenizado[1])

#Quitamos StopWords
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
docs_tokenizado = [[w for w in documento if not w in stop_words] 
             for documento in docs_tokenizado]

print ("\n\nSin stop words\n")
print (docs_tokenizado[1])

[nltk_data] Downloading package punkt to /Users/Mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Numero de documentos: 601

 Enclavado en una situación privilegiada con fantásticas vistas a la sierra, y a menos de 2 Km. del centro de la ciudad de Ronda y 45 Km. de la Costa del Sol, es un lugar ideal para disfrutar de la oferta cultural y gastronómica además del patrimonio histórico-artístico de la ciudad, pudiendo combinarla con actividades en su entorno natural como la Sierra de las Nieves (reserva de la biosfera). Su decoración de estilo moderno y actual ofrece un ambiente selecto y acogedor. En nuestro restaurante podrán degustar los platos de nuestro cocinero Antonio Castro Aguilar además de una exquisita carta de vinos..


Elementos tokenizados

['Enclavado', 'en', 'una', 'situación', 'privilegiada', 'con', 'fantásticas', 'vistas', 'a', 'la', 'sierra', ',', 'y', 'a', 'menos', 'de', '2', 'Km', '.', 'del', 'centro', 'de', 'la', 'ciudad', 'de', 'Ronda', 'y', '45', 'Km', '.', 'de', 'la', 'Costa', 'del', 'Sol', ',', 'es', 'un', 'lugar', 'ideal', 'para', 'disfrutar', 'de', 'la', 'o

**Paso 2** Creación de un un diccionario con todo el vocabulario

In [3]:
diccionario = gensim.corpora.Dictionary(docs_tokenizado)

print("Número de palabras en el diccionario:",len(diccionario))
for i in range(len(diccionario)):
    print(i, diccionario[i])

Número de palabras en el diccionario: 7189
0 albolafia
1 amantes
2 antiguo
3 aves
4 barrio
5 belleza
6 bus
7 calle
8 cama
9 ciudad
10 clientes
11 coche
12 cuna
13 córdoba
14 describir
15 diferentes
16 disposición
17 encuentra
18 especies
19 guadalquivir
20 hermosa
21 hotel
22 información
23 judería
24 mezquita
25 minutos
26 monumentos
27 morena
28 museos
29 naturaleza
30 observar
31 oficina
32 pelo
33 pie
34 posee
35 puede
36 pueden
37 recomendamos
38 relajarnos
39 restaurantes
40 ribera
41 rincón
42 rio
43 secador
44 sierra
45 situados
46 sotos
47 supletoria
48 tabernas
49 tanta
50 televisión
51 tranquilo
52 turismo
53 turísticaestamos
54 típica
55 visitar
56 acogedor
57 actividades
58 actual
59 además
60 aguilar
61 ambiente
62 antonio
63 biosfera
64 carta
65 castro
66 centro
67 cocinero
68 combinarla
69 costa
70 cultural
71 decoración
72 degustar
73 disfrutar
74 enclavado
75 entorno
76 estilo
77 exquisita
78 fantásticas
79 gastronómica
80 históricoartístico
81 ideal
82 km
83 lugar
84

763 agradable
764 alcaicería
765 antigua
766 arco
767 arte
768 asegurar
769 aunque
770 capilla
771 caracterizan
772 cartuja
773 catedral
774 coger
775 colina
776 confort
777 diario
778 diez
779 disfrutando
780 elvira
781 emblemáticos
782 excelente
783 fortaleza
784 fuerzas
785 goza
786 inigualable
787 jardines
788 jerónimo
789 jornadas
790 leyenda
791 localización
792 monasterio
793 necesitan
794 palacios
795 perfecta
796 pies
797 plaza
798 preparadas
799 real
800 sabika
801 san
802 servicio
803 triunfo
804 turísticas
805 ubicación
806 ubicados
807 universitaria
808 abledo
809 abrir
810 amplios
811 andalucía
812 andújar
813 barcafetería
814 beatriz
815 beza
816 cambio
817 capital
818 cazorla
819 cercanía
820 ciudades
821 comida
822 comunican
823 corto
824 cuidadosamente
825 decoradas
826 enclave
827 estrellas
828 fácil
829 geográfico
830 gestión
831 inmejorable
832 jardín
833 jaénrecientemente
834 linares
835 lleves
836 madrid
837 mediterránea
838 mengíbar
839 nosotroscompleta
840 pano

1477 basket
1478 billar
1479 boda
1480 celebrar
1481 chimeneadistintas
1482 comuniones
1483 disfrutarsalón
1484 granadael
1485 güejar
1486 magnificas
1487 nevadaa
1488 padel
1489 sitio
1490 bahía
1491 cable
1492 calefaccion
1493 comunicación
1494 convierten
1495 dotadas
1496 exclusiva
1497 residencial
1498 tranquilas
1499 valdelagrana
1500 activarás
1501 axarquía
1502 benamocarra
1503 brisa
1504 cerro
1505 cima
1506 divisar
1507 duda
1508 familia
1509 igual
1510 jaula
1511 mediterráneo
1512 pareja
1513 respirar
1514 sorprendente
1515 tejeda
1516 ven
1517 vélez
1518 brasa
1519 carnes
1520 dividido
1521 especializado
1522 lumbre
1523 mucha
1524 nazarí
1525 relajarse
1526 rodeados
1527 ventana
1528 zubia
1529 época
1530 actividad
1531 arzobispo
1532 blas
1533 bodega
1534 clásico
1535 combinación
1536 dejará
1537 embargo
1538 extenso
1539 fuensanta
1540 indiferentes
1541 jienenses
1542 lavandería
1543 manjón
1544 municipal
1545 municipio
1546 neomudéjar
1547 niñera
1548 permanece
1549 plan

2226 emplazado
2227 programas
2228 repletas
2229 varios
2230 alcalá
2231 convierte
2232 descansoel
2233 facilidades
2234 geográficael
2235 limítrofe
2236 mantiene
2237 ofrezca
2238 ordenador
2239 orientalesto
2240 perteneciente
2241 portátil
2242 prensa
2243 primeraalcalá
2244 provincias
2245 torrepalma
2246 abre
2247 gaditanos
2248 maría
2249 paisajes
2250 visite
2251 autobus
2252 btt
2253 carrtera
2254 dispuestas
2255 funcionales
2256 mirasol
2257 motril
2258 multitud
2259 quads
2260 sencilla
2261 trayecto
2262 órgiva
2263 caracteriza
2264 cuidada
2265 marco
2266 predomina
2267 santillán
2268 acceder
2269 adicional
2270 coste
2271 cuádruples
2272 dejara
2273 desea
2274 destacan
2275 dream
2276 económico
2277 escapada
2278 especialmente
2279 esquiar
2280 ofrecerá
2281 sencillez
2282 solárium
2283 supermercados
2284 urban
2285 actos
2286 cafébar
2287 casero
2288 climatizados
2289 compartir
2290 conferencias
2291 contiguos
2292 cozazón
2293 cuidado
2294 deseadas
2295 distintos
2296 equi

2976 atendemos
2977 costera
2978 influencia
2979 inglés
2980 jacuzzihidromasaje
2981 córdobaes
2982 disponer
2983 independiente
2984 instalar
2985 repartidas
2986 restauranteel
2987 administración
2988 and
2989 anexos
2990 campanario
2991 chiringuito
2992 chorros
2993 clínica
2994 completan
2995 completos
2996 despachos
2997 duchas
2998 género
2999 oratorio
3000 peluquería
3001 recreativa
3002 rehabilitación
3003 semisótano
3004 subterráneo
3005 ubicadas
3006 viviendas
3007 casablanca
3008 centenar
3009 identidad
3010 momento
3011 polivalente
3012 sacromonte
3013 señas
3014 tapeo
3015 tecnológicas
3016 turista
3017 vacacional
3018 autobusesa
3019 baloncesto
3020 futbol
3021 playaa
3022 polideportivo
3023 alcoba
3024 aquellas
3025 boutiquel
3026 destinado
3027 líneas
3028 racionales
3029 repleto
3030 sostenible
3031 boutique
3032 envolventes
3033 erigida
3034 exaltación
3035 fragancias
3036 poeta
3037 profunda
3038 sonidos
3039 trasladarse
3040 alternar
3041 cadena
3042 conocidos
3043 c

3726 tematizado
3727 temática
3728 tiburón
3729 toalla
3730 tubo
3731 velas
3732 arqueológico
3733 autograph
3734 conceptos
3735 culinarios
3736 existían
3737 fusiona
3738 maestra
3739 paula
3740 r
3741 refectorio
3742 respectivamente
3743 restaurada
3744 sustituyó
3745 xii
3746 banus
3747 maritimo
3748 situción
3749 milla
3750 adecuar
3751 ambiciosa
3752 ampliación
3753 destinados
3754 estándares
3755 existentes
3756 inversión
3757 llevado
3758 marcha
3759 modo
3760 presta
3761 privadas
3762 puesta
3763 restaurant
3764 saladillo
3765 sharing
3766 soul
3767 altas
3768 calidades
3769 complementaria
3770 destacada
3771 diseñados
3772 emplaza
3773 albayzin
3774 clásicas
3775 contemplan
3776 cristiano
3777 extraordinarias
3778 generalife
3779 integran
3780 orden
3781 palació
3782 piezas
3783 ejecutiva
3784 empresas
3785 mujer
3786 sede
3787 barquito
3788 complementarias
3789 flecha
3790 piedras
3791 rompido
3792 acabar
3793 diseña
3794 exótico
3795 fantástica
3796 faro
3797 increíble
3798 

4476 terrazabar
4477 toyo
4478 toyoel
4479 utilización
4480 almenara
4481 elysium
4482 sotogrande
4483 torno
4484 valderrama
4485 alegra
4486 azulejería
4487 corregidor
4488 enrejados
4489 esplendoroso
4490 expone
4491 florida
4492 mestizaje
4493 ocre
4494 plagado
4495 terrazos
4496 alcazaresesta
4497 conexion
4498 estacion
4499 gratuitamente
4500 hotelcomo
4501 justarecientemente
4502 kms
4503 limitadas
4504 pelodispone
4505 satelite
4506 situtado
4507 tftlcd
4508 tipico
4509 facialesel
4510 gratuitael
4511 localel
4512 minibarel
4513 precise
4514 barroco
4515 centros
4516 dolménico
4517 vitalizan
4518 amarga
4519 artesanías
4520 bañeras
4521 cajas
4522 característica
4523 carboneras
4524 diversidad
4525 dosel
4526 encantados
4527 esforzamos
4528 falta
4529 frescas
4530 gestionarlo
4531 jacusi
4532 kiko
4533 lavabos
4534 mira
4535 nijar
4536 nunca
4537 razón
4538 sublime
4539 tio
4540 adaptándose
4541 alimentación
4542 aprovecharse
4543 callejas
4544 cesta
4545 contrastan
4546 cuenco


5225 europalos
5226 frigiliana
5227 gibraltaren
5228 habilitada
5229 idílica
5230 privilegio
5231 torrox
5232 abiertos
5233 asaltan
5234 cabeceros
5235 constante
5236 contemplación
5237 contraposición
5238 corriendo
5239 desapercibido
5240 elaboración
5241 entrando
5242 estructurado
5243 exteriormente
5244 fotografía
5245 frescor
5246 fuerza
5247 hondas
5248 intenso
5249 intenta
5250 jacuzzy
5251 minas
5252 minero
5253 multiusos
5254 murales
5255 poblado
5256 presencia
5257 rodalquilar
5258 selecta
5259 sorpresas
5260 submarinas
5261 trekking
5262 volcánica
5263 kilometros
5264 soleadas
5265 sudoeste
5266 claridaddesde
5267 componen
5268 contribuye
5269 conventotradición
5270 conventoun
5271 cordial
5272 distinto
5273 edificioalgunas
5274 encontrarán
5275 guadalete
5276 homenaje
5277 jefe
5278 mercedarias
5279 peña
5280 poetas
5281 poseen
5282 predominando
5283 querido
5284 señorialesun
5285 transformado
5286 cavana
5287 alturas
5288 árabeandaluz
5289 alejadas
5290 alma
5291 aromas
529

5975 consecutivamente
5976 correos
5977 datos
5978 desk
5979 drago
5980 full
5981 generales
5982 hd
5983 hospitality
5984 hotelhabitaciones
5985 metrostambién
5986 reconocida
5987 secadorinstalaciones
5988 servicesdisponemos
5989 tarda
5990 transmisión
5991 tvlcd
5992 ue
5993 urbanas
5994 viya
5995 wifinuestra
5996 últimos
5997 elviria
5998 roc
5999 apropiado
6000 embarque
6001 sintieran
6002 surgió
6003 tarifeñas
6004 tánger
6005 umbral
6006 vio
6007 adelantos
6008 etcestá
6009 forjada
6010 rl
6011 tecnológicos
6012 úbedacon
6013 com
6014 ecologico
6015 granja
6016 jardin
6017 opcion
6018 romantica
6019 brindando
6020 individualmente
6021 maroma
6022 orquídeas
6023 pantano
6024 romanes
6025 viñuela
6026 desear
6027 santana
6028 ahumado
6029 dotaciones
6030 exe
6031 factura
6032 palmera
6033 albaicin
6034 figura
6035 guardar
6036 guinness
6037 molinos
6038 récords
6039 boliches
6040 construidos
6041 cristalina
6042 experimentará
6043 lindante
6044 vuelta
6045 accesorios
6046 alegres
60

6725 matrimonial
6726 mermeladas
6727 mirando
6728 nocivos
6729 norma
6730 normalmente
6731 obstante
6732 obtenido
6733 orientado
6734 oveja
6735 parecido
6736 posteriores
6737 primitivo
6738 producen
6739 pueblan
6740 quince
6741 recalificación
6742 redonda
6743 registro
6744 renovable
6745 reposar
6746 resguarda
6747 residuos
6748 respecto
6749 reunían
6750 semejantes
6751 semillas
6752 servimos
6753 siguen
6754 sillones
6755 similar
6756 sobriedad
6757 supletorios
6758 sustenta
6759 tantos
6760 tinajas
6761 tocar
6762 trabajadores
6763 transformaron
6764 ultima
6765 uniéndose
6766 utilizadas
6767 vestida
6768 viga
6769 óriginal
6770 arizón
6771 comporta
6772 planalos
6773 base
6774 bondad
6775 canena
6776 castillopalacio
6777 emergen
6778 olivarsus
6779 pino
6780 propiedades
6781 tierraa
6782 atrivm
6783 carro
6784 organizarlas
6785 pozo
6786 pronto
6787 quad
6788 desees
6789 distinguible
6790 laboral
6791 metromar
6792 observarse
6793 pisa
6794 planificar
6795 privilegia
6796 skyli

**Paso 3** Generación del corpus de documentos, se calcula el número de veces que cada palabra aparece en el documento (TF)

In [4]:
corpus = [diccionario.doc2bow(gen_doc) for gen_doc in docs_tokenizado]
print(corpus[67])

[(21, 2), (66, 1), (96, 1), (162, 1), (164, 1), (287, 1), (311, 1), (573, 1), (683, 1), (1240, 1), (2054, 1), (2055, 1), (2056, 1), (2057, 1), (2058, 1), (2059, 1)]


**Paso 4** Generación del modelo para TF-IDF

In [5]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)
tf_idf[corpus[45]]

TfidfModel(num_docs=601, num_nnz=39166)
39166


[(9, 0.03818906765826256),
 (21, 0.007017519648339666),
 (44, 0.12833553278571688),
 (55, 0.08473806387007773),
 (81, 0.11309413793343841),
 (86, 0.06456064374157698),
 (96, 0.0904066592412939),
 (100, 0.06118525128123023),
 (116, 0.045641336075729434),
 (151, 0.12528442992852148),
 (183, 0.05303965194737251),
 (188, 0.13617475055844122),
 (196, 0.06377959427524632),
 (198, 0.07810027055817702),
 (208, 0.06702349777465344),
 (290, 0.038014339556300604),
 (296, 0.057170395569893305),
 (317, 0.39827942149950074),
 (392, 0.10720433948686617),
 (421, 0.06083101841742033),
 (477, 0.0870269193583654),
 (491, 0.09938779865872205),
 (498, 0.07870611968213573),
 (500, 0.10178978509753554),
 (520, 0.0819178524747632),
 (532, 0.06618158071405982),
 (546, 0.12833553278571688),
 (549, 0.1772957126616965),
 (567, 0.10720433948686617),
 (573, 0.09608881955656479),
 (578, 0.06048061517308904),
 (582, 0.1296706151036546),
 (583, 0.1772957126616965),
 (726, 0.09825041934773648),
 (738, 0.091242199324864

**Paso 5** Creación de un índice temporal

In [6]:
print (tempfile.gettempdir())
sims = gensim.similarities.Similarity(tempfile.gettempdir(),tf_idf[corpus],
                                      num_features=len(diccionario))
print (sims[tf_idf[corpus[45]]])

/var/folders/w7/6p2jts811bg78fk9prk3ppr40000gn/T
[2.50748172e-02 4.67274413e-02 5.61061315e-03 1.50708761e-02
 4.91801463e-02 1.04745273e-02 2.41362154e-02 6.59954548e-02
 0.00000000e+00 2.94572264e-02 4.76162434e-02 1.40734226e-01
 6.46158308e-03 3.50262737e-03 2.03444567e-02 4.18958701e-02
 5.14933504e-02 2.20693485e-03 1.78291164e-02 2.28080709e-04
 4.02734727e-02 1.56663004e-02 4.14772741e-02 2.70768777e-02
 8.53183270e-02 1.04364552e-01 5.10980003e-02 8.83291513e-02
 3.23604280e-03 1.38040613e-02 4.02366593e-02 2.01346856e-02
 3.67798544e-02 0.00000000e+00 5.63181862e-02 2.58662701e-02
 3.72334570e-02 5.02345487e-02 3.63293141e-02 5.91653511e-02
 3.01821134e-03 1.89522840e-02 4.72832061e-02 4.33607809e-02
 1.80975571e-02 9.99999940e-01 1.93825066e-02 6.51541203e-02
 8.34397785e-03 1.39104128e-02 2.70864274e-02 1.81327143e-03
 5.38292155e-02 5.73998243e-02 3.42336074e-02 2.63522156e-02
 2.02940255e-02 6.90183043e-03 2.02929508e-02 1.19473413e-02
 6.09480310e-03 3.39296348e-02 4.567

**Paso 6** Obtener elementos recomendados, los más similares

In [7]:
salida = sims[tf_idf[corpus[10]]]

heap = []
indice = 0
for medida in salida:
    heapq.heappush(heap, (medida, documentos[indice]))
    indice +=1

print (documentos[10])
heapq.nlargest(10,heap)

El Hotel Pequeño Rocío es un complejo hotelero que consta de dos zonas visiblemente diferenciadas, las zonas comunes y la zona de alojamiento. La zona de alojamiento dispone de 22 casas rurales, dichas casas mantienen la fisonomía típica del lugar (La aldea Del Rocío), hecho que permite que nuestros clientes se sientan totalmente integrados en el entorno, manteniendo una decoración individual y personalizada. Casa Standard: Cada una de ellas tiene un porche de entrada y un patio trasero, amplio salón comedor totalmente equipado (TV, cómodos sofás, climatiación, etc.), un cuarto de baño con ducha térmica, secador de pelo y todos los complementos necesarios para el aseo, dos dormitorios cada uno de ellos para dos personas. Todas las casas disponen de calefacción y aire acondicionado en todas las dependencias. Las casas tienen una capacidad para 4 Pax (sin incluir camas supletorias). Suites: Las suites tienen todas las comodidades de nuestras casas standard añadiéndoles más espacio en cad

[(0.99999994,
  'El Hotel Pequeño Rocío es un complejo hotelero que consta de dos zonas visiblemente diferenciadas, las zonas comunes y la zona de alojamiento. La zona de alojamiento dispone de 22 casas rurales, dichas casas mantienen la fisonomía típica del lugar (La aldea Del Rocío), hecho que permite que nuestros clientes se sientan totalmente integrados en el entorno, manteniendo una decoración individual y personalizada. Casa Standard: Cada una de ellas tiene un porche de entrada y un patio trasero, amplio salón comedor totalmente equipado (TV, cómodos sofás, climatiación, etc.), un cuarto de baño con ducha térmica, secador de pelo y todos los complementos necesarios para el aseo, dos dormitorios cada uno de ellos para dos personas. Todas las casas disponen de calefacción y aire acondicionado en todas las dependencias. Las casas tienen una capacidad para 4 Pax (sin incluir camas supletorias). Suites: Las suites tienen todas las comodidades de nuestras casas standard añadiéndoles m

## Nuevas técnicas, deep learning

Es una nueva técnica dentro del aprendizaje automático, basado en la estructura de las redes neuronales, pero usando arquitecturas profundas (deep) en la que múltiples capas son conectadas.

La principal característica y ventaja del deep learning es el **feature discovering** sin la necesidad de la intervención humana.

### Paragraph vector

- Algoritmo de aprendizaje no supervisado capaz de generar una representacion vectorial a partir de texto.
- Paragraph vector es capaz de aprender la semántica de las palabras.

<img src="./recursos/doc2vec.jpg">

#### Ejemplo

**Paso 1** Entrenamiento del modelo

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Configuración
#-------------
__longitud_vector = 50
__ventana = 10
__frecuencia_minima = 5
__learning_rate = 0.025
__min_learning_rate = 0.025
__epoch = 100
#---------------------

                
tagged_data = [TaggedDocument(words=word_tokenize(_d .lower()), tags=[str(i)]) for i, _d in enumerate(documentos)]
model = gensim.models.Doc2Vec(vector_size=__longitud_vector, window=__ventana, min_count=__frecuencia_minima, workers=11,alpha=__learning_rate, min_alpha=__min_learning_rate, epochs=__epoch)
model.build_vocab(tagged_data)
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

#Recorrido de todos los documentos con los que se ha entrenado
for i in range(20):
    print (model.docvecs[i].tolist())
    


[1.587451696395874, -1.7444400787353516, 1.3544639348983765, -1.7673488855361938, 2.462223768234253, 3.2801802158355713, 0.7250093221664429, 0.11332249641418457, -1.809435486793518, -0.7094475626945496, 4.326848983764648, -4.671313285827637, -1.630766749382019, 0.7918123006820679, -2.493673801422119, -1.1639457941055298, -0.912265419960022, -0.6189419627189636, -1.6819876432418823, -0.10107243061065674, 0.20826879143714905, 0.3045748472213745, 2.930406332015991, -0.8769654631614685, 1.1216562986373901, 0.6692726016044617, -5.4294962882995605, 4.889317035675049, 2.6082241535186768, -0.9663776755332947, -2.3446974754333496, -1.5982508659362793, -3.295801877975464, -0.10403364151716232, 2.986062526702881, 3.3235719203948975, -0.9251510500907898, 2.987150192260742, -1.7619761228561401, -3.7785136699676514, 0.7216004729270935, -2.3025264739990234, 0.6477640271186829, -1.1343470811843872, -2.557004690170288, -2.80895733833313, 0.9745473861694336, -0.8781617283821106, 0.017737800255417824, -2

**Paso 2** Cálculo de similitud

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

elemento =model.docvecs[100]

heap = []
for i in range(len(model.docvecs)):
    sim =cosine_similarity(elemento.reshape(1, -1), model.docvecs[i].reshape(1, -1))
    heapq.heappush(heap, (sim, documentos[i]))

print (documentos[100])
heapq.nlargest(10,heap)

Este bonito aparthotel se encuentra en la hermosa localidad de Zahara de los Atunes. Se trata de un complejo de apartamentos turísticos que ofrecen los servicios de un hotel. Ideal para un público con espí­ritu joven, ofrece estudios con literas, estudios con camas dobles, cunas, cocina equipada, microondas, baño privado con plato de ducha y secador, TV, DVD, calefacción, aire acondicionado y acceso a internet. Todo a muy buen precio, con un ambiente moderno y acogedor para que disfrute de la belleza de Cádiz y de las fabulosas playas de Zahara de los Atunes.


[(array([[0.99999994]], dtype=float32),
  'Este bonito aparthotel se encuentra en la hermosa localidad de Zahara de los Atunes. Se trata de un complejo de apartamentos turísticos que ofrecen los servicios de un hotel. Ideal para un público con espí\xadritu joven, ofrece estudios con literas, estudios con camas dobles, cunas, cocina equipada, microondas, baño privado con plato de ducha y secador, TV, DVD, calefacción, aire acondicionado y acceso a internet. Todo a muy buen precio, con un ambiente moderno y acogedor para que disfrute de la belleza de Cádiz y de las fabulosas playas de Zahara de los Atunes.'),
 (array([[0.6212976]], dtype=float32),
  'El Hotel Best Sabinal tiene una ubicación privilegiada, situado en una de las playas más bonitas de la costa de Almería y a la vez en el centro de Roquetas de Mar.'),
 (array([[0.58484423]], dtype=float32),
  'Este hotel vacacional se encuentra en la Urbanización El Toyo, frente al Mar Mediterráneo, a diez minutos en coche de la entrada sur 

### Inferir vector para un nuevo texto
Una vez tenemos el modelo podemos inferir el vector para un nuevo texto, a continuación realizaremos un ejemplo

In [10]:
text = ["comer","marisco"]

#Obtener el texto de un nuevo vector
vector = model.infer_vector(text)
print(vector)

[ 0.09529391 -0.05151384  0.312698   -0.3053242  -0.4592347  -0.45407885
 -0.00086174  0.42636585  0.2522933   0.06088032  0.3934628  -0.03262608
 -0.51093847 -0.32650253 -0.522195   -0.0503957  -0.05778503 -0.14930099
 -0.00954633 -0.26136592 -0.14014597  0.12913592 -0.02551915 -0.01382646
 -0.48161912  0.05429152 -0.35100237  0.10069957  0.8069769  -0.14649306
 -0.2250238  -0.32851335 -0.17979729  0.06312641  0.3532753  -0.10011166
 -0.19286853 -0.12393226 -0.32169554 -0.4212818   0.3301665  -0.48805213
 -0.3376467  -0.2789674   0.595198   -0.6718742   0.2831881  -0.42944673
 -0.1203654  -0.5554792 ]
